Lambda School Data Science

*Unit 2, Sprint 2, Module 3*

---

# Cross-Validation


## Assignment
- [ ] [Review requirements for your portfolio project](https://lambdaschool.github.io/ds/unit2), then submit your dataset.
- [ ] Continue to participate in our Kaggle challenge. 
- [ ] Use scikit-learn for hyperparameter optimization with RandomizedSearchCV.
- [ ] Submit your predictions to our Kaggle competition. (Go to our Kaggle InClass competition webpage. Use the blue **Submit Predictions** button to upload your CSV file. Or you can use the Kaggle API to submit your predictions.)
- [ ] Commit your notebook to your fork of the GitHub repo.


You won't be able to just copy from the lesson notebook to this assignment.

- Because the lesson was ***regression***, but the assignment is ***classification.***
- Because the lesson used [TargetEncoder](https://contrib.scikit-learn.org/categorical-encoding/targetencoder.html), which doesn't work as-is for _multi-class_ classification.

So you will have to adapt the example, which is good real-world practice.

1. Use a model for classification, such as [RandomForestClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html)
2. Use hyperparameters that match the classifier, such as `randomforestclassifier__ ...`
3. Use a metric for classification, such as [`scoring='accuracy'`](https://scikit-learn.org/stable/modules/model_evaluation.html#common-cases-predefined-values)
4. If you’re doing a multi-class classification problem — such as whether a waterpump is functional, functional needs repair, or nonfunctional — then use a categorical encoding that works for multi-class classification, such as [OrdinalEncoder](https://contrib.scikit-learn.org/categorical-encoding/ordinal.html) (not [TargetEncoder](https://contrib.scikit-learn.org/categorical-encoding/targetencoder.html))



## Stretch Goals

### Reading
- Jake VanderPlas, [Python Data Science Handbook, Chapter 5.3](https://jakevdp.github.io/PythonDataScienceHandbook/05.03-hyperparameters-and-model-validation.html), Hyperparameters and Model Validation
- Jake VanderPlas, [Statistics for Hackers](https://speakerdeck.com/jakevdp/statistics-for-hackers?slide=107)
- Ron Zacharski, [A Programmer's Guide to Data Mining, Chapter 5](http://guidetodatamining.com/chapter5/), 10-fold cross validation
- Sebastian Raschka, [A Basic Pipeline and Grid Search Setup](https://github.com/rasbt/python-machine-learning-book/blob/master/code/bonus/svm_iris_pipeline_and_gridsearch.ipynb)
- Peter Worcester, [A Comparison of Grid Search and Randomized Search Using Scikit Learn](https://blog.usejournal.com/a-comparison-of-grid-search-and-randomized-search-using-scikit-learn-29823179bc85)

### Doing
- Add your own stretch goals!
- Try other [categorical encodings](https://contrib.scikit-learn.org/categorical-encoding/). See the previous assignment notebook for details.
- In additon to `RandomizedSearchCV`, scikit-learn has [`GridSearchCV`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html). Another library called scikit-optimize has [`BayesSearchCV`](https://scikit-optimize.github.io/notebooks/sklearn-gridsearchcv-replacement.html). Experiment with these alternatives.
- _[Introduction to Machine Learning with Python](http://shop.oreilly.com/product/0636920030515.do)_ discusses options for "Grid-Searching Which Model To Use" in Chapter 6:

> You can even go further in combining GridSearchCV and Pipeline: it is also possible to search over the actual steps being performed in the pipeline (say whether to use StandardScaler or MinMaxScaler). This leads to an even bigger search space and should be considered carefully. Trying all possible solutions is usually not a viable machine learning strategy. However, here is an example comparing a RandomForestClassifier and an SVC ...

The example is shown in [the accompanying notebook](https://github.com/amueller/introduction_to_ml_with_python/blob/master/06-algorithm-chains-and-pipelines.ipynb), code cells 35-37. Could you apply this concept to your own pipelines?


### BONUS: Stacking!

Here's some code you can use to "stack" multiple submissions, which is another form of ensembling:

```python
import pandas as pd

# Filenames of your submissions you want to ensemble
files = ['submission-01.csv', 'submission-02.csv', 'submission-03.csv']

target = 'status_group'
submissions = (pd.read_csv(file)[[target]] for file in files)
ensemble = pd.concat(submissions, axis='columns')
majority_vote = ensemble.mode(axis='columns')[0]

sample_submission = pd.read_csv('sample_submission.csv')
submission = sample_submission.copy()
submission[target] = majority_vote
submission.to_csv('my-ultimate-ensemble-submission.csv', index=False)
```

In [1]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Kaggle-Challenge/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'

In [11]:
import pandas as pd

# Merge train_features.csv & train_labels.csv
train = pd.merge(pd.read_csv(DATA_PATH+'waterpumps/train_features.csv'), 
                 pd.read_csv(DATA_PATH+'waterpumps/train_labels.csv'))

# Read test_features.csv & sample_submission.csv
test = pd.read_csv(DATA_PATH+'waterpumps/test_features.csv')
sample_submission = pd.read_csv(DATA_PATH+'waterpumps/sample_submission.csv')

In [3]:
train.shape, test.shape

((59400, 41), (14358, 40))

In [431]:
train.loc[train['population'] < 5, 'population'] = 0
#train.loc[train['population'] > 15000, 'population'] = 0

In [12]:
import numpy as np
def wrangle(X):
  X = X.copy()
  X['latitude'] = X['latitude'].replace(-2e-08, 0)
  cols_with_zeros = ['longitude', 'latitude', 'construction_year', 'gps_height', 'population']
  for col in cols_with_zeros:
    X[col] = X[col].replace(0, np.nan)
#    X[col + '_Missing'] = X[col].isnull()
 
  
  cols_to_drop = ['id', 'quantity_group', 'recorded_by']
  X = X.drop(columns = cols_to_drop)
  X['date_recorded'] = pd.to_datetime(X['date_recorded'], infer_datetime_format = True)
  X['year_recorded'] = X['date_recorded'].dt.year
  X['month_recorded'] = X['date_recorded'].dt.month
  X['day_recorded'] = X['date_recorded'].dt.day
  X = X.drop(columns = 'date_recorded')

  X['years'] = X['year_recorded'] - X['construction_year']
#  years = X['years']
#  for year in years:
#    if year < 0:
#      year = 0
#    else:
#      return year
  
#  X['year_Missing'] = X['years'].isnull()
  return X

train = wrangle(train)
#val = wrangle(val)
test = wrangle(test)


In [5]:
from math import radians, cos, sin, asin, sqrt

def location(lon, lat, degrees=True):
    
    r = 6371 # Earth's radius (km). Have r = 3956 if you want miles

    # Convert decimal degrees to radians
    if degrees:
        lon, lat = map(radians, [lon, lat])

    # 'Single-point' Haversine formula
    a = sin(lon/2)**2 + cos(lon) * sin(lat/2)**2
    d = 2 * r * asin(sqrt(a)) 

    return d
train['location'] = [location(longitude, latitude) for longitude, latitude in zip(train.longitude, train.latitude)]
test['location'] = [location(longitude, latitude) for longitude, latitude in zip(test.longitude, test.latitude)]

In [6]:
train = train.drop(['longitude', 'latitude'], axis = 1)
test = test.drop(['longitude', 'latitude'], axis = 1)
train.loc[train['years'] < 0, 'years'] = 0
train['years'] = train['years'].replace(0, np.nan)
train['missing_years'] = train['years'].isnull()
test.loc[test['years'] < 0, 'years'] = 0
test['years'] = test['years'].replace(0, np.nan)
test['missing_years'] = test['years'].isnull()

In [413]:
train.head()

,amount_tsh,funder,gps_height,installer,wpt_name,num_private,basin,subvillage,region,region_code,...,source_type,source_class,waterpoint_type,waterpoint_type_group,status_group,year_recorded,month_recorded,day_recorded,years,location
0,6000.0,Roman,1390.0,Roman,none,0,Lake Nyasa,Mnyusi B,Iringa,11,...,spring,groundwater,communal standpipe,communal standpipe,functional,2011,3,14,12.0,4017.577520
1,0.0,Grumeti,1399.0,GRUMETI,Zahanati,0,Lake Victoria,Nyamara,Mara,20,...,rainwater harvesting,surface,communal standpipe,communal standpipe,functional,2013,3,6,3.0,3864.784165
2,25.0,Lottery Club,686.0,World vision,Kwa Mahundi,0,Pangani,Majengo,Manyara,21,...,dam,surface,communal standpipe multiple,communal standpipe,functional,2013,2,25,4.0,4183.886745
3,0.0,Unicef,263.0,UNICEF,Zahanati Ya Nanyumbu,0,Ruvuma / Southern Coast,Mahakamani,Mtwara,90,...,borehole,groundwater,communal standpipe multiple,communal standpipe,non functional,2013,1,28,27.0,4428.683943
4,0.0,Action In A,NaN,Artisan,Shuleni,0,Lake Victoria,Kyanyamisa,Kagera,18,...,rainwater harvesting,surface,communal standpipe,communal standpipe,functional,2011,7,13,NaN,3466.941219


In [13]:
target = 'status_group'
train_features = train.drop(columns = [target])
numeric_features = train_features.select_dtypes(include = 'number').columns.tolist()
cardinality = train_features.select_dtypes(exclude = 'number').nunique()
categorical_features = cardinality[cardinality <= 50].index.tolist()
features = numeric_features + categorical_features

In [14]:
from sklearn.model_selection import train_test_split
train, val = train_test_split(train, train_size = 0.80, test_size = 0.20, 
                              stratify = train['status_group'], random_state = 42)

In [15]:
target = 'status_group'
X_train = train[features]
y_train = train[target]
X_val = val[features]
y_val = val[target]
X_test = test[features]

In [16]:
import category_encoders as ce 
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline

pipeline = make_pipeline(
    ce.OneHotEncoder(use_cat_names = True),
    SimpleImputer(strategy = 'median'),
    RandomForestClassifier(random_state = 42, n_estimators = 100, n_jobs = -1)
)
pipeline.fit(X_train, y_train)
print('Train: ', pipeline.score(X_train, y_train))
print('Validatiion: ', pipeline.score(X_val, y_val))

Train:  0.9979166666666667
Validatiion:  0.8086700336700336


In [17]:
target = 'status_group'
X_train = train.drop(columns = target)
y_train = train[target]
X_val = val.drop(columns = target)
y_val = val[target]
X_test = test

In [18]:
pipeline = make_pipeline(
    ce.OrdinalEncoder(),
    SimpleImputer(strategy = 'mean'),
    RandomForestClassifier(random_state = 0, n_estimators = 100, n_jobs = -1)
)
pipeline.fit(X_train, y_train)
print('Train: ', pipeline.score(X_train, y_train))
print('Validatiion: ', pipeline.score(X_val, y_val))

Train:  0.9999579124579124
Validatiion:  0.8085016835016835


In [398]:
import pandas_profiling
pandas_profiling.__version__

'2.4.0'

In [19]:
import category_encoders as ce

from sklearn.feature_selection import f_regression, SelectKBest
from sklearn.impute import SimpleImputer
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor

In [20]:
k = 3
scores = cross_val_score(pipeline, X_train, y_train, cv = k, 
                        scoring = 'accuracy')
print(f'accuracy score {k} folds: ', scores)

accuracy score 3 folds:  [0.79804293 0.80915404 0.79463384]


In [423]:
from scipy.stats import randint, uniform

In [424]:
from sklearn.model_selection import RandomizedSearchCV
pipeline = make_pipeline(
    ce.OrdinalEncoder(),
    SimpleImputer(),
    RandomForestClassifier(random_state = 42)
)
param_distrib = {
    'simpleimputer__strategy': ['median', 'mean'],
    'randomforestclassifier__n_estimators' : randint(200, 400),
    'randomforestclassifier__max_depth' : [10, 15, None],
    'randomforestclassifier__max_features': uniform(0, 1),
}

In [425]:
search = RandomizedSearchCV(
    pipeline,
    param_distributions = param_distrib,
    n_iter = 8,
    cv = 3,
    scoring = 'accuracy',
    verbose = 10,
    return_train_score = True,
    n_jobs = 7
)
search.fit(X_train, y_train)

Fitting 3 folds for each of 8 candidates, totalling 24 fits


[Parallel(n_jobs=7)]: Using backend LokyBackend with 7 concurrent workers.
[Parallel(n_jobs=7)]: Done   4 tasks      | elapsed:  2.1min
[Parallel(n_jobs=7)]: Done  11 tasks      | elapsed:  5.4min
[Parallel(n_jobs=7)]: Done  14 out of  24 | elapsed:  6.6min remaining:  4.7min
[Parallel(n_jobs=7)]: Done  17 out of  24 | elapsed:  9.0min remaining:  3.7min
[Parallel(n_jobs=7)]: Done  20 out of  24 | elapsed: 10.0min remaining:  2.0min
[Parallel(n_jobs=7)]: Done  24 out of  24 | elapsed: 13.3min finished


RandomizedSearchCV(cv=3, error_score='raise-deprecating',
                   estimator=Pipeline(memory=None,
                                      steps=[('ordinalencoder',
                                              OrdinalEncoder(cols=None,
                                                             drop_invariant=False,
                                                             handle_missing='value',
                                                             handle_unknown='value',
                                                             mapping=None,
                                                             return_df=True,
                                                             verbose=0)),
                                             ('simpleimputer',
                                              SimpleImputer(add_indicator=False,
                                                            copy=True,
                                                            f

In [437]:
print('param', search.best_params_)
print('score', search.best_score_)

param {'randomforestclassifier__max_depth': None, 'randomforestclassifier__max_features': 0.19577163902114625, 'randomforestclassifier__n_estimators': 380, 'simpleimputer__strategy': 'median'}
score 0.7985058922558923


In [438]:
from sklearn.metrics import accuracy_score
pipeline = search.best_estimator_
val_pred = pipeline.predict(X_val)
score = accuracy_score(y_val, val_pred)
print(score)

ValueError: Unexpected input dimension 33, expected 40

In [428]:
test_predict = pipeline.predict(X_test)
test_id = sample_submission['id']
data = {'ID': test_id, 'status_group': test_predict}
cv_pred = pd.DataFrame(data)
cv_pred

,ID,status_group
0,50785,functional
1,51630,functional
2,17168,functional
3,45559,non functional
4,49871,functional
...,...,...
14353,39307,non functional
14354,18990,functional
14355,28749,functional
14356,33492,functional


In [333]:
cv_pred.to_csv('CV_prediction.csv', index = False)